# Single facet simulation

TODO
 - Section 1 - costruct a displacement controlled model and show the load displacement curve for
   sliding
 - Section 2 - with the identified 

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model

In [5]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

In [6]:
n_e = 1
L_x = 1.0
xd_inner = XDomainFEGrid(coord_min=(0, -1),
                          coord_max=(1, 0),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_outer = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_inner.mesh.I[-1, :]

(-1, slice(None, None, None))

In [7]:
m1 = MATS2DElastic(E=28000, nu=0.3)
m2 = MATS2DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_inner.mesh.I[-1, :],
    J=xd_outer.mesh.I[0, :],
    fets=FETS1D52ULRH()
)

In [8]:
material_params = dict(
    E_T=10000, gamma_T=4000, K_T=0, S_T=0.01, c_T=2, bartau=5, 
    E_N=25000, S_N=0.05, c_N = 1, m = 0.008, f_t=5, f_c=150, f_c0=100, eta=0.0 # 0.5
)
bond_m = Slide34(**material_params)

In [9]:
m = TStepBC(
    domains=[
        (xd_inner, m1),
        (xd_outer, m2),
        (xd12, bond_m),
    ],
)

Note that the enumeration of `dofs` within a serialized `fe_domain` is done 
only after the domains have been registered within the `TStepBC`. Therefore,
final numbers of `dofs` are available only here and not before constructing `TStepBC`.
Whan manipulating the `dofs` individually - like here - when we want to 
construct nodal loads, the boundary conditions must be constructed once
the subdomains `xd_left`, `xd_right` and `xd12`  have been registered within the `TStepBC`.

Since `BCSlice` cannot be used currently with the 
natural boundary conditions we need to apply the nodal load to the individual degrees of freedom. To get them, let us misuse BCSlice as a quick access to the dofs. This can be 
done also directly by accessing the `mesh` object @RC - TODO

Control boundary conditions are now defined as nodal values. Define the time function with on unloading cycle.

In [10]:
tf = TFCyclicNonsymmetricConstant(number_of_cycles=2, unloading_ratio=0.2)
tf.interact()

To find out subcritical value of force, run first a displacement control simulation, find out the maximum value.
Then divide the obtained peak load with the number of control degrees of freedom, i.e. 4 in our case.

In [11]:
load_dim = 0 # 
fixed_dim = 1 # 0 - x, 1 - y
right_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
right_control_dofs = right_control_slice.dofs
right_control_dofs

array([ 8, 10, 12, 14])

In [12]:
disp_control = True
if disp_control:
    right_control = [BCDof(var='u', dof=dof, value=0.4 ) 
                     for dof in right_control_dofs ]
else:
    F_max = 45 / len(right_control_dofs)
    right_control = [BCDof(var='f', dof=dof, value=F_max, time_function=tf ) 
                     for dof in right_control_dofs ]

Fix the remaining boundary conditions

In [13]:
right_fixed = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                 var='u', dims=[fixed_dim], value=0.0)
left_fixed = BCSlice(slice=xd_inner.mesh[:, :, :, :],
                 var='u', dims=[0,1], value=0)
bc1 = right_control + [right_fixed, left_fixed]

In [14]:
m.bc=bc1

In [15]:
m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s = m.sim
s.tloop.verbose = True # set false to avoid the prints during each increment
s.tloop.k_max = 50
s.tline.step = 0.01

In [16]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.01

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
F_to = m.hist.F_t
U_to = m.hist.U_t
right_control_dofs = [bc.dof for bc in right_control]
F_l_t = np.sum(F_to[:, right_control_dofs], axis=-1)
U_r_t = np.average(U_to[:, right_control_dofs], axis=-1)
_, ax = plt.subplots(1,1)
ax.plot(U_r_t, F_l_t)

In [ ]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [ ]:
w_pi, s_pi_x, s_pi_y, z, alpha_x, alpha_y, omega_T, omega_N, sig_pi, tau_pi_x, tau_pi_y, \
Z, X_x, X_y, Y_T, Y_N = np.zeros((16, len(U_r_t)))

for i in range(len(m.hist.state_vars)): 
    w_pi[i] = m.hist.state_vars[i][2]['w_pi'][0,0]
    s_pi_x[i] = m.hist.state_vars[i][2]['s_pi_x'][0,0]
    s_pi_y[i] = m.hist.state_vars[i][2]['s_pi_y'][0,0]
    z[i] = m.hist.state_vars[i][2]['z'][0,0]
    alpha_x[i] = m.hist.state_vars[i][2]['alpha_x'][0,0]
    omega_T[i] = m.hist.state_vars[i][2]['omega_T'][0,0]
    omega_N[i] = m.hist.state_vars[i][2]['omega_N'][0,0]
    sig_pi[i] = m.hist.state_vars[i][2]['sig_pi'][0,0]
    tau_pi_x[i] = m.hist.state_vars[i][2]['tau_pi_x'][0,0]
    tau_pi_y[i] = m.hist.state_vars[i][2]['tau_pi_y'][0,0]
    Z[i] = m.hist.state_vars[i][2]['Z'][0,0]
    X_x[i] = m.hist.state_vars[i][2]['X_x'][0,0]
    X_y[i] = m.hist.state_vars[i][2]['X_y'][0,0]
    Y_T[i] = m.hist.state_vars[i][2]['Y_T'][0,0]
    Y_N[i] = m.hist.state_vars[i][2]['Y_N'][0,0]
    
_, (ax1) = plt.subplots(1,1)
ax1.plot(U_r_t, np.abs(sig_pi), label='confinement')
ax1.plot(U_r_t, np.abs(tau_pi_x), label='shear')
#ax2.plot(U_r_t, omega_T, label='tangential damage')
plt.legend()



In [ ]:
#m.hist.state_vars

In [ ]:
sigma_N